# GCP Storage Tutorial

**Workflow:** Source notebook.

**Objective:** Provide sample code for uploading and downloading different file types.

In [1]:
import os, sys, json
import pandas as PD

In [2]:
import io, base64
import numpy as NP
import cv2 as CV
import google.cloud.storage as GS

Database Host: 34.74.56.48


In [4]:

DATA_PATH = os.path.join(UT.ROOT_PATH, 'source_apis', 'data')

In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [6]:
# setup the GCP client 

CREDENTIALS = os.path.join(BACKEND_PATH, 'app_proj', 'common', 'gcp-cred-injurycheck1.json')
storageClient = GS.Client.from_service_account_json(CREDENTIALS)

### Text 

In [7]:
# setup the bucket path

BUCKET = 'source-apis-2019'
bucket = storageClient.bucket(BUCKET)
items = list(storageClient.list_blobs(BUCKET, prefix='elitehrv'))
items = [t.name for t in items]

print( '\n'.join(items) )

elitehrv/all-athletes-2019.csv


In [8]:
# download file data

for cloudPath in items:
    print(cloudPath)
    
    blob = bucket.blob(cloudPath)
    hrvBt = blob.download_as_string()
    
    stream = io.BytesIO(hrvBt)
    exampleDf = PD.read_csv(stream)

exampleDf.head()

elitehrv/all-athletes-2019.csv


,Member,Type,Date Time Start,Date Time End,Duration,HRV,Morning Readiness,Balance,HRV CV,HR,lnRmssd,Rmssd,Nn50,Pnn50,Sdnn,Low Frequency Power,High Frequency Power,LF/HF Ratio,Total Power
0,012065014,open,2019-09-23 06:54:45,NaN,53.417,77,NaN,NaN,0.0,57.68,4.97337,144.51,36,71,139.12,2256.370,9729.246,0.232,11985.616
1,012065014,open,2019-09-25 06:56:09,NaN,52.097,80,NaN,NaN,0.0,58.74,5.19765,180.85,40,78,176.74,17301.231,4135.452,4.184,21436.684
2,012065014,open,2019-09-27 06:56:53,NaN,42.169,80,NaN,NaN,0.0,74.34,5.23139,187.05,38,73,268.82,6072.220,4735.696,1.282,10807.915
3,012065014,readiness,2019-10-04 07:04:37,NaN,110.815,74,NaN,NaN,0.0,72.98,4.80254,121.82,71,53,149.81,16425.251,7177.288,2.289,23602.539
4,012065014,readiness,2019-10-07 06:38:42,NaN,91.959,75,8.0,PS,0.0,58.34,4.84465,127.06,60,67,186.36,1712.849,4202.184,0.408,5915.033


### Image

In [9]:
# download frame images to local disk

bucketName = 'umd-athletics'
dateTaken = '2020-05-21'
subBucket = 'front'

bucket = storageClient.bucket(bucketName)
frameModels = VM.Frame.objects.filter(Bucket=bucketName, DateTaken=dateTaken, SubBucket=subBucket)
print(len(list(frameModels)))

for idx, frm in enumerate(frameModels):
    # get data from cloud

    cloudPath = f"{frm.DateTaken}/{frm.SubBucket}/{frm.Name}"
    blob = bucket.blob(cloudPath)
    imageTx = blob.download_as_string()

    # save data as image to disk

    nparr = NP.frombuffer(imageTx, NP.uint8)
    img = CV.imdecode(nparr, CV.IMREAD_COLOR)
    outputPath = os.path.join(DATA_PATH, frm.Name) 
    results = CV.imwrite(outputPath, img)

    # for dev only
    if idx > 5: break

2452
